# Implementing Pandas with GIS Data

The ArcGIS API for python gives us this flexability!

In [ ]:
import arcpy
import arcgis
import pandas as pd

### Transforming local GIS Data to a Pandas Dataframe

In [ ]:
ca_city_boundary = 'ca_cities_boundaries'
sdf = pd.DataFrame.spatial.from_featureclass(ca_city_boundary)

In [ ]:
sdf

Now we can manipulate the table with Pandas!

In [ ]:
sdf['Pop2010'].mean()

In [ ]:
avg = sdf['Pop2010'].mean()
sdf.loc[sdf['Pop2010'] > avg]

In [ ]:
# TIP, we can also run the above query using the .query method
# sdf.query('Pop2010 > @avg')

In [ ]:
sdf = sdf.loc[sdf['Pop2010'] > avg]

Now lets save all of the cities that have a population greater than the average <br>
as a new feature class!

In [ ]:
arcpy.env.addOutputsToMap = False

output_fc = os.path.join(arcpy.env.workspace, "larger_cities")
sdf.spatial.to_featureclass(output_fc, sanitize_columns=False)

We can also export this data to csv or excel! Whatever we need. <br>
However, since there is spatial data in the shape column, it makes sense for us to hold on to that and convert to a feature class

### Getting a Rest Service as a dataframe

Previously we created feature layers on ArcGIS online using the API <br>
How can we interact with them as dataframes?

Lets say you needed to issue a report of all of the earthquakes in the world for the last 30 days.<br>
And this report needed to be recreated at the end of every month<br>
Luckily ESRI serves a rest service of earthquakes that is updated continously<br>
In the output report, we only want earthquakes with magnitude greater than or equal to 5, and we want to clean the table a little.<br>

In [ ]:
mygis = arcgis.GIS('pro')

In [ ]:
earthquakes = r'https://services9.arcgis.com/RHVPKKiFTONKtxq3/ArcGIS/rest/services/USGS_Seismic_Data_v1/FeatureServer/0'

earthquakes_fl = arcgis.features.FeatureLayer(earthquakes)

In [ ]:
earthquakes_fl.query(as_df=True)

In [ ]:
earthquakes_df = earthquakes_fl.query(where="mag >= 5", as_df=True)

In [ ]:
earthquakes_df = earthquakes_df[['id', 'mag', 'place', 'eventTime', 
                                 'dmin', 'longitude', 'latitude', 'SHAPE']]

In [ ]:
earthquakes_df

In [ ]:
output_fc = os.path.join(arcpy.env.workspace, 'Earthquakes_thismonth')
earthquakes_df.spatial.to_featureclass(output_fc, sanitize_columns=False)

In [ ]:
earthquakes_df.drop(columns=['SHAPE'], inplace=True)
earthquakes_df.to_excel('Earthquakes_thismonth.xlsx', index=False, sheet_name='from_python')